In [ ]:
import os
from dotenv import load_dotenv
import requests
from pprint import pprint
# Load environment variables from .env file
load_dotenv()

rapidapi_key = os.getenv('RAPIDAPI_KEY')
# API call Search by location https://rapidapi.com/3b-data-3b-data-default/api/airbnb13/

url = "https://airbnb13.p.rapidapi.com/search-location"

# querystring = {"location":"Vancouver","checkin":"2025-01-12","checkout":"2025-01-13","adults":"2","children":"0","infants":"0",
               "pets":"0","page":"2","currency":"CAD"}

headers = {
	"x-rapidapi-key": rapidapi_key,
	"x-rapidapi-host": "airbnb13.p.rapidapi.com",
	"Content-Type": "application/json"
}
response = requests.get(url, headers=headers, params=querystring)
pprint(response.json())